In [112]:
import pandas as pd
import requests
import getpass

In [113]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pandas columns
pd.set_option('display.max_columns', None)

In [115]:
countries = [
    'Belgium','Germany','Denmark','Estonia','Spain','Finland','France',
'Ireland','Lithuania','Latvia','Netherlands','Poland','Portugal','Sweden',
'United Kingdom'
]

Data comes from [IUCN Red List](https://www.iucnredlist.org/search?searchType=species) after filtering by Systems=Marine


In [210]:
# We get the species by country
countriesMarine = pd.read_csv('../data/marineIUCN/countries.csv')
countriesMarine = countriesMarine[countriesMarine.name.isin(countries)]
# The taxonomy so we can filter the marine fishes 
# Read here for more information https://portals.iucn.org/library/node/45188
taxonomy = pd.read_csv('../data/marineIUCN/taxonomy.csv')
fishesTaxo = taxonomy[taxonomy.className.isin(['CHONDRICHTHYES', 'ACTINOPTERYGII', 'MYXINI'])]
countriesMarine = fishesTaxo.merge(countriesMarine, on='scientificName')
# Then get the species classification (Vulnerable, Endangered, etc.)
classification = pd.read_csv('../data/marineIUCN/simple_summary.csv')
countriesMarineClass = countriesMarine.merge(classification, on='scientificName',how='inner', suffixes=('', '_y'))
countriesMarineClass.drop(countriesMarineClass.filter(regex='_y$').columns, axis=1, inplace=True)
# We calculate the percentage of threatened species by country
categories = ['Critically Endangered','Endangered',
       'Least Concern', 'Lower Risk/least concern', 'Near Threatened',
       'Vulnerable', 'Data Deficient']
threatened = ['Critically Endangered','Endangered',
              'Vulnerable']
threatenedCountry = countriesMarineClass[countriesMarineClass.redlistCategory.isin(threatened)].groupby(['name','redlistCategory']).size().reset_index(name='counts').groupby(['name']).sum(numeric_only=True)
totalCountry = countriesMarineClass[countriesMarineClass.redlistCategory.isin(categories)].groupby(['name','redlistCategory']).size().reset_index(name='counts').groupby(['name']).sum(numeric_only=True)
# Inverse to make a score 0-100 with 0 being the worst
threatenedPct = 100 - threatenedCountry/totalCountry * 100
threatenedPct.rename(columns={'counts':'2023'}, inplace=True)
threatenedPct

,2023
name,
Belgium,84.967320
Denmark,86.138614
Estonia,90.243902
Finland,90.000000
France,91.332611
Germany,82.894737
Ireland,91.885965
Latvia,88.636364
Lithuania,88.372093


In [ ]:
tokenIUCN = getpass.getpass('IUCN Red List Token: ')

In [237]:
# Get historical assessment classification from IUCN Red List
# THIS TAKES 26 minutes

# speciesName = countriesMarineClass.scientificName.unique().tolist() 
# speciesHistory = pd.DataFrame()
# for species in speciesName:
#     response = requests.get("https://apiv3.iucnredlist.org/api/v3/species/history/name/{0}?token={1}".format(species,tokenIUCN))
#     tempDf = pd.DataFrame.from_dict(response.json()['result'])
#     tempDf['scientific_name'] = species
#     speciesHistory = pd.concat([speciesHistory,tempDf],axis=0)
# speciesHistory.to_csv('../data/marineFishEUhistoryIUCN.csv', index=False)

speciesHistory = pd.read_csv('../data/marineFishEUhistoryIUCN.csv')
speciesHistory

,year,assess_year,code,category,scientific_name
0,2020,2019,NT,Near Threatened,Hexanchus griseus
1,2009,2005,NT,Near Threatened,Hexanchus griseus
2,2000,2000,LR/nt,Lower Risk/near ...,Hexanchus griseus
3,1996,1996,VU,Vulnerable,Hexanchus griseus
4,2019,2018,VU,Vulnerable,Lamna nasus
...,...,...,...,...,...
1740,2020,2018,EN,Endangered,Mobula mobular
1741,2019,2018,EN,Endangered,Mobula mobular
1742,2022,2019,EN,Endangered,Mobula birostris
1743,2020,2019,EN,Endangered,Mobula birostris


## Using API

This includes 201 fresh water species. No way to filter AFAIK

In [204]:
# Get list ISO codes as used in the IUCN Red List API

response = requests.get("https://apiv3.iucnredlist.org/api/v3/country/list?token={0}".format(tokenIUCN))
countryList = pd.DataFrame.from_dict(response.json()['results'])
countryList.loc[
    countryList["country"].str.contains(r"^(?=.*United)(?=.*Kingdom)"), "country"
] = "United Kingdom"
countriesISO = countryList[countryList.country.isin(countries)]['isocode'].to_list()

In [126]:
# get species per country
countrySpecies = pd.DataFrame()
for country in countriesISO:
    response = requests.get("https://apiv3.iucnredlist.org/api/v3/country/getspecies/{0}?token={1}".format(country, tokenIUCN))
    tempDf = pd.DataFrame.from_dict(response.json()['result'])
    tempDf['country'] = country
    countrySpecies = pd.concat([countrySpecies, tempDf], axis=0)

In [183]:
# Get list of species in Europe with classification (Vulnerable, Endangered, etc.)
pageNum = list(range(0,16))
species = pd.DataFrame()
for pageNum in pageNum:
    response = requests.get("https://apiv3.iucnredlist.org/api/v3/species/page/{0}?token={1}".format(pageNum,tokenIUCN))
    tempDf = pd.DataFrame.from_dict(response.json()['result'])
    species = pd.concat([species,tempDf],axis=0)

In [202]:
# filter Marine species as per https://portals.iucn.org/library/node/45188
speciesMarine = species[species.class_name.isin(['CHONDRICHTHYES', 'ACTINOPTERYGII', 'MYXINI'])]
countriesMarine = countrySpecies.merge(speciesMarine, on='scientific_name',how='inner', suffixes=('', '_y'))
countriesMarine.drop(countriesMarine.filter(regex='_y$').columns, axis=1, inplace=True)
# We calculate the percentage of threatened species by country
categories = ['VU', 'CR', 'LC', 'EN', 'DD', 'NT', 'NA', 'EX', 'EW']
threatened = ['VU', 'CR','EN', 'EX', 'EW']
totalCountry = countriesMarine[countriesMarine.category.isin(categories)].groupby(['country','category']).size().reset_index(name='counts').groupby(['country']).sum(numeric_only=True)
threatenedCountry = countriesMarine[countriesMarine.category.isin(threatened)].groupby(['country','category']).size().reset_index(name='counts').groupby(['country']).sum(numeric_only=True)

# Inverse to make a score 0-100 with 0 being the worst
threatenedPct = 100 - threatenedCountry/totalCountry * 100
threatenedPct.rename(columns={'counts':'2023'}, inplace=True)
threatenedPct

,2023
country,
BE,85.128205
DE,77.822581
DK,86.008230
EE,94.285714
ES,89.991372
FI,93.670886
FR,90.178571
GB,87.521079
IE,89.205703
